In [3]:
import pandas as pd

from core.operation import *

In [5]:
current_date = date.today().strftime('%Y-%m-%d')

In [7]:
# Get Stock Data
ml_ret = pd.read_parquet(get_live_stock() / 'trade_stock_ml_ret.parquet.brotli')
ml_trend = pd.read_parquet(get_live_stock() / 'trade_stock_ml_trend.parquet.brotli')
port_iv = pd.read_parquet(get_live_stock() / 'trade_stock_port_iv.parquet.brotli')
port_im = pd.read_parquet(get_live_stock() / 'trade_stock_port_im.parquet.brotli')
port_id = pd.read_parquet(get_live_stock() / 'trade_stock_port_id.parquet.brotli')
port_ivm = pd.read_parquet(get_live_stock() / 'trade_stock_port_ivm.parquet.brotli')
trend_mls = pd.read_parquet(get_live_stock() / 'trade_stock_trend_mls.parquet.brotli')
mrev_etf = pd.read_parquet(get_live_stock() / 'trade_stock_mrev_etf.parquet.brotli')
mrev_mkt = pd.read_parquet(get_live_stock() / 'trade_stock_mrev_mkt.parquet.brotli')

# Load Live Price
permno_data = pd.read_parquet(get_live_price() / 'data_permno_live.parquet.brotli')
mrev_etf_hedge_data = pd.read_parquet(get_live_price() / 'data_mrev_etf_hedge_live.parquet.brotli')
mrev_mkt_hedge_data = pd.read_parquet(get_live_price() / 'data_mrev_mkt_hedge_live.parquet.brotli')
trend_mls_com_data = pd.read_parquet(get_live_price() / 'data_trend_mls_com_live.parquet.brotli')
trend_mls_bond_data = pd.read_parquet(get_live_price() / 'data_trend_mls_bond_live.parquet.brotli')
ml_trend_re_data = pd.read_parquet(get_live_price() / 'data_ml_trend_re_live.parquet.brotli')
ml_trend_bond_data = pd.read_parquet(get_live_price() / 'data_ml_trend_bond_live.parquet.brotli')

# Merge data by 'date', 'ticker', 'type'
stock_data = pd.concat([ml_ret, ml_trend, port_iv, port_im, port_id, port_ivm, trend_mls, mrev_etf, mrev_mkt], axis=0)
stock_data = stock_data.groupby(level=['date', 'ticker', 'type']).sum()
stock_data = stock_data.loc[stock_data.index.get_level_values('date') == current_date]

# Merge price data
permno_data = permno_data.reset_index().set_index(['ticker', 'date'])
price_data = pd.concat([permno_data, mrev_etf_hedge_data, mrev_mkt_hedge_data, trend_mls_bond_data, trend_mls_com_data, ml_trend_re_data, ml_trend_bond_data], axis=0)
price_data = price_data.loc[~price_data.index.duplicated(keep='last')]

In [10]:
'IYR' in stock_data.index.get_level_values('ticker')

True

In [11]:
stock_data

weight
date       ticker type           
2024-02-09 A      long   0.001905
           AA     short  0.000243
           AAP    long   0.001905
           ABBV   long   0.002670
           ADSK   short  0.001905
...                           ...
           XLV    short  0.002305
           XLY    short  0.001651
           XOP    long   0.003221
           XYL    short  0.000205
           ZTO    short  0.000244

[216 rows x 1 columns]

In [13]:
'IYR' in permno_data.index.get_level_values('ticker')

False